In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
from io import StringIO

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context
from airflow.models import Variable

In [2]:
year = 1994 + hash(f'e-korobkova') % 23

In [3]:
year

2005

In [4]:
default_args = {
    'owner': 'e-korobkova',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2023, 1, 26),
    'schedule_interval': '0 12 * * *'
}

In [17]:
#link ='/var/lib/airflow/airflow.git/dags/a.batalov/vgsales.csv'

In [5]:
vgsales = pd.read_csv('vgsales.csv')

In [21]:
vgsales

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [6]:
vgsales = vgsales.query('Year == @year')

In [23]:
vgsales

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
24,25,Grand Theft Auto: Vice City,PS2,2002.0,Action,Take-Two Interactive,8.41,5.49,0.47,1.78,16.15
25,26,Pokemon Ruby/Pokemon Sapphire,GBA,2002.0,Role-Playing,Nintendo,6.06,3.90,5.38,0.50,15.85
114,115,Medal of Honor: Frontline,PS2,2002.0,Shooter,Electronic Arts,2.93,2.75,0.17,0.99,6.83
131,132,Kingdom Hearts,PS2,2002.0,Role-Playing,Sony Computer Entertainment,3.64,1.20,1.49,0.07,6.40
135,136,Super Mario Sunshine,GC,2002.0,Platform,Nintendo,4.01,1.26,0.87,0.17,6.31
...,...,...,...,...,...,...,...,...,...,...,...
16426,16429,ESPN NBA 2Night 2002,XB,2002.0,Sports,Konami Digital Entertainment,0.01,0.00,0.00,0.00,0.01
16464,16467,Egg Mania: Eggstreme Madness,GC,2002.0,Puzzle,Kemco,0.01,0.00,0.00,0.00,0.01
16556,16559,Super Robot Taisen: Original Generation,GBA,2002.0,Role-Playing,Banpresto,0.01,0.00,0.00,0.00,0.01
16563,16566,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,0.00,0.01


In [ ]:
#Какая игра была самой продаваемой в этом году во всем мире?

In [24]:
the_best_game = vgsales[vgsales['Global_Sales'] == vgsales['Global_Sales'].max()]['Name']

In [25]:
the_best_game

24    Grand Theft Auto: Vice City
Name: Name, dtype: object

In [ ]:
#Игры какого жанра были самыми продаваемыми в Европе? Перечислить все, если их несколько

In [31]:
genres = vgsales.groupby('Genre', as_index = False).agg({'EU_Sales': 'sum'})

In [32]:
genres

,Genre,EU_Sales
0,Action,27.01
1,Adventure,2.51
2,Fighting,6.21
3,Misc,3.56
4,Platform,12.30
5,Puzzle,1.41
6,Racing,8.89
7,Role-Playing,9.39
8,Shooter,14.98
9,Simulation,3.53


In [38]:
the_best_genre = genres[genres['EU_Sales']== genres['EU_Sales'].max()]['Genre']

In [39]:
the_best_genre

0    Action
Name: Genre, dtype: object

In [ ]:
# На какой платформе было больше всего игр, которые продались более чем миллионным тиражом в Северной Америке?

In [40]:
vgsales_NA_million = vgsales.query('NA_Sales > 1.0')

In [42]:
vgsales_NA_million.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
24,25,Grand Theft Auto: Vice City,PS2,2002.0,Action,Take-Two Interactive,8.41,5.49,0.47,1.78,16.15
25,26,Pokemon Ruby/Pokemon Sapphire,GBA,2002.0,Role-Playing,Nintendo,6.06,3.90,5.38,0.50,15.85
114,115,Medal of Honor: Frontline,PS2,2002.0,Shooter,Electronic Arts,2.93,2.75,0.17,0.99,6.83
131,132,Kingdom Hearts,PS2,2002.0,Role-Playing,Sony Computer Entertainment,3.64,1.20,1.49,0.07,6.40
135,136,Super Mario Sunshine,GC,2002.0,Platform,Nintendo,4.01,1.26,0.87,0.17,6.31


In [43]:
platforms = vgsales_NA_million.groupby('Platform', as_index = False).agg({'NA_Sales': 'count'})

In [44]:
the_best_platforms= platforms[platforms['NA_Sales']== platforms['NA_Sales'].max()]['Platform']

In [45]:
platforms

,Platform,NA_Sales
0,GBA,7
1,GC,6
2,PC,3
3,PS,1
4,PS2,27
5,XB,6


In [46]:
the_best_platforms

4    PS2
Name: Platform, dtype: object

In [47]:
#У какого издателя самые высокие средние продажи в Японии?

In [48]:
vgsales

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
24,25,Grand Theft Auto: Vice City,PS2,2002.0,Action,Take-Two Interactive,8.41,5.49,0.47,1.78,16.15
25,26,Pokemon Ruby/Pokemon Sapphire,GBA,2002.0,Role-Playing,Nintendo,6.06,3.90,5.38,0.50,15.85
114,115,Medal of Honor: Frontline,PS2,2002.0,Shooter,Electronic Arts,2.93,2.75,0.17,0.99,6.83
131,132,Kingdom Hearts,PS2,2002.0,Role-Playing,Sony Computer Entertainment,3.64,1.20,1.49,0.07,6.40
135,136,Super Mario Sunshine,GC,2002.0,Platform,Nintendo,4.01,1.26,0.87,0.17,6.31
...,...,...,...,...,...,...,...,...,...,...,...
16426,16429,ESPN NBA 2Night 2002,XB,2002.0,Sports,Konami Digital Entertainment,0.01,0.00,0.00,0.00,0.01
16464,16467,Egg Mania: Eggstreme Madness,GC,2002.0,Puzzle,Kemco,0.01,0.00,0.00,0.00,0.01
16556,16559,Super Robot Taisen: Original Generation,GBA,2002.0,Role-Playing,Banpresto,0.01,0.00,0.00,0.00,0.01
16563,16566,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,0.00,0.01


In [50]:
vgsales_publishers = vgsales.groupby('Publisher', as_index = False).agg({'JP_Sales': 'mean'})

In [51]:
the_best_publishers = vgsales_publishers[vgsales_publishers['JP_Sales']== vgsales_publishers['JP_Sales'].max()]['Publisher']

In [52]:
vgsales_publishers

,Publisher,JP_Sales
0,3DO,0.000000
1,505 Games,0.250000
2,Acclaim Entertainment,0.000000
3,Activision,0.002400
4,Activision Value,0.000000
...,...,...
94,Vivendi Games,0.003750
95,Wanadoo,0.060000
96,XS Games,0.000000
97,Xicat Interactive,0.000000


In [53]:
the_best_publishers

63    Nintendo
Name: Publisher, dtype: object

In [54]:
#Сколько игр продались лучше в Европе, чем в Японии?

In [7]:
best_in_EU = vgsales[vgsales['EU_Sales'] > vgsales['JP_Sales']].shape[0]

In [8]:
best_in_EU

694